In [1]:
from pringles.simulator import Simulator
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

In [2]:
for atomic in mySimulator.atomic_registry.discovered_atomics:
    if atomic.__name__ == "Server":
        Server = atomic
    elif atomic.__name__ == "AutoScaler":
        AutoScaler = atomic
    elif atomic.__name__ == "ServerQueue":
        ServerQueue = atomic
    elif atomic.__name__ == "Dispatcher":
        Dispatcher = atomic

print(Server)
print(Dispatcher)

<class 'pringles.models.models.Server'>
<class 'pringles.models.models.Dispatcher'>


In [3]:
from pringles.models import Coupled 
a_dispatcher = Dispatcher("a_dispatcher", numberOfServers=5, server2="free")

print("Inport names: ", [port.name for port in a_dispatcher.inports])
print("Outport names: ", [port.name for port in a_dispatcher.outports])

Inport names:  ['newJob', 'jobDone', 'serverStackInfo']
Outport names:  ['requestJob', 'server0', 'server1', 'server2']


In [12]:
top_model = (Coupled(name='top', subcomponents=[a_dispatcher])
                .add_inport("newJob")
                .add_inport("jobDone")             
                .add_inport("serverStackInfo")
                .add_outport("requestJob")
                .add_outport("server0")
                .add_outport("server1")
                .add_outport("server2") 
                .add_coupling('newJob', a_dispatcher.get_port("newJob"))
                .add_coupling('jobDone', a_dispatcher.get_port("jobDone"))
                .add_coupling('serverStackInfo', a_dispatcher.get_port("serverStackInfo"))
             
                .add_coupling(a_dispatcher.get_port('requestJob'), "requestJob")
                .add_coupling(a_dispatcher.get_port('server0'), "server0")
                .add_coupling(a_dispatcher.get_port('server1'), "server1")
                .add_coupling(a_dispatcher.get_port('server2'), "server2")
            )
top_model

In [10]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

In [13]:
sim_events = [
    Event(VirtualTime(0,0,20,0,0), top_model.get_port('serverStackInfo'), [float(2),float(0)]),
    Event(VirtualTime(0,0,25,0,0), top_model.get_port('newJob'), float(1)),
    Event(VirtualTime(0,0,35,0,0), top_model.get_port('serverStackInfo'), [float(3),float(1)]),
    Event(VirtualTime(0,0,40,0,0), top_model.get_port('serverStackInfo'), [float(1),float(1)]),
    Event(VirtualTime(0,0,42,0,0), top_model.get_port('newJob'), float(1))
]

a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(50), 
                          events=sim_events,
                          working_dir='sim_results/'
                         )

results = mySimulator.run_simulation(a_simulation)

In [14]:
print(results.get_process_output())

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from sim_results/2019-10-09-180845-7a262f0b6a1d44f78218975616483ae8/top_model
Loading events from sim_results/2019-10-09-180845-7a262f0b6a1d44f78218975616483ae8/events
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Mess

In [15]:
display(results.output_df.head(100))

,time,port,value
0,00:00:00:000,requestjob,1.0
1,00:00:40:000,requestjob,1.0
2,00:00:42:000,server1,1.0


In [16]:
print(results.logs_dfs.keys(),'\n\n')
display(results.logs_dfs['ParallelRoot'].head())

dict_keys(['a_dispatcher', 'top', 'ParallelRoot']) 




,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,Y,00:00:00:000,top(02),requestjob,1.0,ParallelRoot(00)
1,0,L,Y,00:00:40:000,top(02),requestjob,1.0,ParallelRoot(00)
2,0,L,Y,00:00:42:000,top(02),server1,1.0,ParallelRoot(00)


In [11]:
# import matplotlib.pyplot as plt
# from pringles.utils import vtime_decorate

# fig, axes = plt.subplots(1, 3, figsize=(20, 10))
# results.plot_port('ParallelRoot', 'out_port', axes=vtime_decorate(axes[0]), index=0)
# results.plot_port('ParallelRoot', 'out_port', axes=vtime_decorate(axes[1]), index=1)
# results.plot_port('ParallelRoot', 'out_port', axes=vtime_decorate(axes[2]), index=2)

In [ ]:
pickle_path = a_simulation.output_dir + '/simulation.pkl'

recovered_simulation = Simulation.read_pickle(pickle_path)

**Warning:** for this to work you will have to have the Simulator object instanciated so the model Atomics are discovered.

In [ ]:
recovered_simulation.result.output_df.head()

In [ ]:
results.plot_port('newJob')